In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

In [2]:
#データセットの読み込み
#MNISTデータの読み込み
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

#訓練データとテストデータの取得
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels


#訓練データとテストデータのサイズ確認
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.
(55000, 784)
(55000, 10)
(10000, 784)
(10000, 10)


In [3]:
#モデルを設定
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 400)
        self.prelu1 = nn.PReLU()
        self.fc2 = nn.Linear(400, 400)
        self.prelu2 = nn.PReLU()
        self.fc3 = nn.Linear(400, 100)
        self.prelu3 = nn.PReLU()
        self.fc4 = nn.Linear(100, 100)
        self.prelu4 = nn.PReLU()
        self.fc5 = nn.Linear(100, 50)
        self.prelu5 = nn.PReLU()
        self.fc6 = nn.Linear(50, 50)
        self.prelu6 = nn.PReLU()
        self.fc7 = nn.Linear(50, 20)
        self.prelu7 = nn.PReLU()
        self.fc8 = nn.Linear(20, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.prelu1(self.fc1(x))
        x = self.prelu2(self.fc2(x))
        x = self.prelu3(self.fc3(x))
        x = self.prelu4(self.fc4(x))
        x = self.prelu5(self.fc5(x))
        x = self.prelu6(self.fc6(x))
        x = self.prelu7(self.fc7(x))
        x = F.softmax(self.fc8(x), dim=1)
        return x

In [4]:
#GPUが使えるかどうかの確認
use_cuda = torch.cuda.is_available()

In [5]:
#GPUでモデルを訓練するように設定
model = Net().cuda()

In [6]:
# 損失関数と最適化手法の設定
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [7]:
#合成データの作成
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

#データセットの作成
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)


In [8]:
#学習データの読み込み器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#テストデータの読み込み器
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [9]:
#学習
for epoch in range(100):  
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        if use_cuda:
            inputs, labels = inputs.cuda(), labels.cuda()

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels.float())  
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))

        #ファストストックを設定
            
        if loss.item() < 1.48:
            break
    if epoch > 50:  
        if loss.item() < 1.465:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))
            print('Finished Training')
            break
        
print('Finished Training')

[1,     1] loss: 2.302
[1,    11] loss: 2.302
[1,    21] loss: 2.231
[1,    31] loss: 2.216
[1,    41] loss: 2.138
[1,    51] loss: 2.033
[1,    61] loss: 2.078
[1,    71] loss: 1.939
[1,    81] loss: 1.944
[1,    91] loss: 1.868
[1,   101] loss: 1.932
[1,   111] loss: 1.872
[1,   121] loss: 1.857
[1,   131] loss: 1.881
[1,   141] loss: 1.843
[1,   151] loss: 1.878
[1,   161] loss: 1.787
[1,   171] loss: 1.849
[1,   181] loss: 1.675
[1,   191] loss: 1.835
[1,   201] loss: 1.838
[1,   211] loss: 1.737
[1,   221] loss: 1.822
[1,   231] loss: 1.878
[1,   241] loss: 1.759
[1,   251] loss: 1.873
[1,   261] loss: 1.848
[1,   271] loss: 1.749
[1,   281] loss: 1.804
[1,   291] loss: 1.664
[1,   301] loss: 1.746
[1,   311] loss: 1.725
[1,   321] loss: 1.684
[1,   331] loss: 1.655
[1,   341] loss: 1.630
[1,   351] loss: 1.745
[1,   361] loss: 1.649
[1,   371] loss: 1.672
[1,   381] loss: 1.650
[1,   391] loss: 1.649
[1,   401] loss: 1.628
[1,   411] loss: 1.676
[1,   421] loss: 1.588
[1,   431] 

In [10]:
#学習したモデルでテストデータを予測
outputs = model(x_test.cuda())
_, predicted = torch.max(outputs.data, 1)
#予測結果をONE-HOT表現に変換
predicted = predicted.cpu().numpy()
import numpy 
predicted = numpy.eye(10)[predicted]
print(predicted)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
#正解率の計算
from sklearn.metrics import accuracy_score
print("テストデータの予測正解率は:",accuracy_score(y_test, predicted))

テストデータの予測正解率は: 0.938
